In [1]:
# setup import
import pandas as pd 
import requests
import wikipedia as wp
from bs4 import BeautifulSoup

import io

from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
# getting data from internet
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")


In [3]:
# creat a new Dataframe
df = pd.read_html(html, header = 0)[0]
df = df[df.Borough != 'Not assigned']


In [4]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

In [5]:
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

In [6]:
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [7]:
df.shape

(103, 3)

In [8]:
geo_data="http://cocl.us/Geospatial_data"
geo_data=requests.get(geo_data).content
df_temp=pd.read_csv(io.StringIO(geo_data.decode('utf-8')))

In [9]:
# rename the first column to allow merging dataframes on Postcode
df_temp.columns = ['Postcode', 'Latitude', 'Longitude']
df = pd.merge(df_temp, df, on='Postcode')

In [10]:
# reorder column names and show the dataframe
df = df[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]

In [11]:
# load latitude and longitude into geopandas dataframe
df['Coordinates'] = list(zip(df.Longitude, df.Latitude))
df['Coordinates'] = df['Coordinates'].apply(Point)
geo_df = gpd.GeoDataFrame(df, geometry='Coordinates')

In [12]:
geo_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Coordinates
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,POINT (-79.19435 43.80669)
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,POINT (-79.16050 43.78454)
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,POINT (-79.18871 43.76357)
3,M1G,Scarborough,Woburn,43.770992,-79.216917,POINT (-79.21692 43.77099)
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,POINT (-79.23948 43.77314)
...,...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188,POINT (-79.51819 43.70688)
99,M9P,Etobicoke,Westmount,43.696319,-79.532242,POINT (-79.53224 43.69632)
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724,POINT (-79.55472 43.68891)
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437,POINT (-79.58844 43.73942)
